In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyvi

     |████████████████████████████████| 8.5MB 2.9MB/s 
     |████████████████████████████████| 747kB 38.9MB/s 


#Model mới

In [ ]:
import re
import os
import numpy as np
import math
from pyvi import ViTokenizer, ViPosTagger
import pickle

def files_contents_words(path):
    files=[os.path.join(path, f) for f in os.listdir(path)]
    contents=[]
    words=set()
    for file in files:
        with open(file,encoding="utf8") as f:
            string=f.read()
            string=re.sub(r'[^-/\w\s]','',string).split()
            contents.append(string)
            words.update(string)
    words=list(words)
    return files, contents, words

def calc_tf_idf(words, contents):       
    model=dict()
    N=len(contents)
    idf_vector=[]
    for w in words:
        model[w]=dict()
        for i,d in enumerate(contents):
            c=d.count(w)
            # tf
            if c:
                tf=c/len(d)
                model[w][i]=tf
        #idf
        idf=1+math.log10(N/len(model[w]))
        idf_vector.append(idf)
        
    #tf-idf
    for i,w in enumerate(model.keys()):
        for d in model[w].keys():
            model[w][d]*=idf_vector[i]
    return model,idf_vector

#Norm2
def Norm2(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            res.append((i,np.linalg.norm(vt-vector_query))) #
        res.sort(key=lambda x: x[1])
        return res
    except:
        return None

#Cosine
def Cosine(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            cos=(vt@vector_query)/(np.linalg.norm(vt)*np.linalg.norm(vector_query))
            res.append((i,cos))
        res.sort(reverse=True,key=lambda x: x[1])
        return res
    except:
        return None

#Scalar
def Scalar(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            SProduct=vt@vector_query
            res.append((i,SProduct))
        res.sort(reverse=True,key=lambda x: x[1])
        return res[:5]
    except:
        return None

path='/content/drive/MyDrive/CS521/DaXuLi1'
files, contents, words=files_contents_words(path)
model,idf_vector=calc_tf_idf(words, contents)

### Pickle Model
print("Processing __")

word_path = '/content/drive/MyDrive/CS521/Vector Space/word.pkl'
pickle.dump(words, open(word_path, "wb"))
print('Word saved')

contents_path = '/content/drive/MyDrive/CS521/Vector Space/contents.pkl'
pickle.dump(contents, open(contents_path, "wb"))
print('Contents saved')

vector_path = '/content/drive/MyDrive/CS521/Vector Space/tfidf.pkl'
pickle.dump(idf_vector, open(vector_path, "wb"))
print("TF-IDF Saved")

model_path = '/content/drive/MyDrive/CS521/Vector Space/model.pkl'
pickle.dump(model, open(model_path, "wb"))
print("Vector Space model saved")

Processing __
Word saved
Contents saved
TF-IDF Saved
Vector Space model saved


#Model cũ

In [ ]:
import re
import os
import numpy as np
import math
from pyvi import ViTokenizer, ViPosTagger
import pickle

def files_contents_words(path):
    files=[os.path.join(path, f) for f in os.listdir(path)]
    contents=[]
    words=set()
    for file in files:
        with open(file,encoding="utf8") as f:
            string=f.read()
            string=re.sub(r'[^-/\w\s]','',string).split()
            contents.append(string)
            words.update(string)
    words=list(words)
    return files, contents, words

def calc_tf_idf(words, contents):       
    model=dict()
    N=len(contents)
    idf_vector=[]
    for w in words:
        model[w]=dict()
        for i,d in enumerate(contents):
            c=d.count(w)
            # tf
            if c:
                tf=c/len(d)
                model[w][i]=tf
        #idf
        idf=1+math.log10(N/len(model[w]))
        idf_vector.append(idf)
        
    #tf-idf
    for i,w in enumerate(model.keys()):
        for d in model[w].keys():
            model[w][d]*=idf_vector[i]
    return model,idf_vector

#Norm2
def Norm2(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            res.append((i,np.linalg.norm(vt-vector_query))) #
        res.sort(key=lambda x: x[1])
        return res
    except:
        return None

#Cosine
def Cosine(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            cos=(vt@vector_query)/(np.linalg.norm(vt)*np.linalg.norm(vector_query))
            res.append((i,cos))
        res.sort(reverse=True,key=lambda x: x[1])
        return res
    except:
        return None

#Scalar
def Scalar(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            SProduct=vt@vector_query
            res.append((i,SProduct))
        res.sort(reverse=True,key=lambda x: x[1])
        return res[:5]
    except:
        return None

path='/content/drive/MyDrive/CS521/LuatDatDai2013_DaXuLi'
files, contents, words=files_contents_words(path)
model,idf_vector=calc_tf_idf(words, contents)

### Pickle Model
print("Processing __")

word_path = '/content/drive/MyDrive/CS521/Vector Space/word_old.pkl'
pickle.dump(words, open(word_path, "wb"))
print('Word saved')

contents_path = '/content/drive/MyDrive/CS521/Vector Space/contents_old.pkl'
pickle.dump(contents, open(contents_path, "wb"))
print('Contents saved')

vector_path = '/content/drive/MyDrive/CS521/Vector Space/tfidf_old.pkl'
pickle.dump(idf_vector, open(vector_path, "wb"))
print("TF-IDF Saved")

model_path = '/content/drive/MyDrive/CS521/Vector Space/model_old.pkl'
pickle.dump(model, open(model_path, "wb"))
print("Vector Space model saved")

Processing __
Word saved
Contents saved
TF-IDF Saved
Vector Space model saved


#Test Model cũ

In [ ]:
import pickle
from pyvi import ViTokenizer, ViPosTagger
import numpy as np

def load_model():
    file_path = ["/content/drive/MyDrive/CS521/Vector Space/model_old.pkl", "/content/drive/MyDrive/CS521/Vector Space/tfidf_old.pkl", "/content/drive/MyDrive/CS521/Vector Space/word_old.pkl"]
    ### Load Model
    with open(file_path[0], 'rb') as f:
        model = pickle.load(f)
    ### Load Idf_vector
    with open(file_path[1], 'rb') as f:
        idf_vector = pickle.load(f)
    ### Load Word
    with open(file_path[2], 'rb') as f:
        word = pickle.load(f)    
    return model, idf_vector, word

def preprocessing_query(word):
    dict_path =r'/content/drive/MyDrive/CS521/pro_dictionary.txt'
    with open(dict_path, encoding='utf8') as fobj:
	    stop_words=fobj.read()

    words = ViTokenizer.tokenize(word)
    words = words.lower()
    words = words.split()
    stop_words = stop_words.split()
    prew = ''
    for word in words:
        if word not in stop_words:
            prew = prew + word + ' '

    return prew

def Norm2(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            res.append((i,np.linalg.norm(vt-vector_query))) #
        res.sort(key=lambda x: x[1])
        return res[:5]
    except:
        return None

#Scalar
def Scalar(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            SProduct=vt@vector_query
            res.append((i,SProduct))
        res.sort(reverse=True,key=lambda x: x[1])
        return res[:5]
    except:
        return None
        
#Cosine
def Cosine(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            cos=(vt@vector_query)/(np.linalg.norm(vt)*np.linalg.norm(vector_query))
            res.append((i,cos))
        res.sort(reverse=True,key=lambda x: x[1])
        return res[:5]
    except:
        return None

def print_doc(res):
    for doc in res:
    #print(doc[0])
        #id = ''
        #if doc[0] < 10:
            #id = '' + str(doc[0])
        #else:
            #id = str(doc[0])
        id=doc[0]
        id=id+1
        
        print("id document: ",id)
        id1=str(id)
        ori_doc_path = "/content/drive/MyDrive/CS521/LuatDatDai2013/Dieu"+id1+'.txt'
        #print(path)
        
        with open(ori_doc_path, 'r', encoding="utf8") as f:
            full_doc = f.read()
        print(full_doc[:200])
        print("...................................................")
        
model, idf_vector, words = load_model()
#print(words)
query = "sở hữu"
print("QUERY: ", query )
query = preprocessing_query(query)
print("After processing", query)
print("------------------------------------------------\n Kết quả tìm kiếm:")
res = Cosine(model,words,idf_vector,query)
#print(res)
if res != None:
    print(res)
    print_doc(res)

QUERY:  sở hữu
After processing sở_hữu 
------------------------------------------------
 Kết quả tìm kiếm:
[(1, 1.0), (136, 1.0), (9, 1.0), (10, 1.0), (137, 1.0)]
id document:  2
Chương 1.
QUY ĐỊNH CHUNG
Điều 2. Đối tượng áp dụng
1. Cơ quan nhà nước thực hiện quyền hạn và trách nhiệm đại diện chủ sở hữu toàn dân về đất đai, thực hiện nhiệm vụ thống nhất quản lý nhà nước về đất
...................................................
id document:  137
Chương 10.
CHẾ ĐỘ SỬ DỤNG CÁC LOẠI ĐẤT
MỤC 2. ĐẤT NÔNG NGHIỆP
Điều 137. Đất rừng đặc dụng
1. Nhà nước giao đất rừng đặc dụng cho tổ chức quản lý rừng đặc dụng để quản lý, bảo vệ theo quy hoạch, kế hoạ
...................................................
id document:  10
Chương 1.
QUY ĐỊNH CHUNG
Điều 10. Phân loại đất
Căn cứ vào mục đích sử dụng, đất đai được phân loại như sau:
1. Nhóm đất nông nghiệp bao gồm các loại đất sau đây:
a) Đất trồng cây hàng năm gồm đất trồ
...................................................
id document:  11
Chương 1.

In [ ]:
import pickle
from pyvi import ViTokenizer, ViPosTagger
import numpy as np

def load_model():
    file_path = ["/content/drive/MyDrive/CS521/Vector Space/model.pkl", "/content/drive/MyDrive/CS521/Vector Space/tfidf.pkl", "/content/drive/MyDrive/CS521/Vector Space/word.pkl"]
    ### Load Model
    with open(file_path[0], 'rb') as f:
        model = pickle.load(f)
    ### Load Idf_vector
    with open(file_path[1], 'rb') as f:
        idf_vector = pickle.load(f)
    ### Load Word
    with open(file_path[2], 'rb') as f:
        word = pickle.load(f)    
    return model, idf_vector, word

def preprocessing_query(word):
    dict_path =r'/content/drive/MyDrive/CS521/pro_dictionary.txt'
    with open(dict_path, encoding='utf8') as fobj:
	    stop_words=fobj.read()

    words = ViTokenizer.tokenize(word)
    words = words.lower()
    words = words.split()
    stop_words = stop_words.split()
    prew = ''
    for word in words:
        if word not in stop_words:
            prew = prew + word + ' '

    return prew

def Norm2(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            res.append((i,np.linalg.norm(vt-vector_query))) #
        res.sort(key=lambda x: x[1])
        return res[:5]
    except:
        return None

#Scalar
def Scalar(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            SProduct=vt@vector_query
            res.append((i,SProduct))
        res.sort(reverse=True,key=lambda x: x[1])
        return res[:5]
    except:
        return None
        
#Cosine
def Cosine(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            cos=(vt@vector_query)/(np.linalg.norm(vt)*np.linalg.norm(vector_query))
            res.append((i,cos))
        res.sort(reverse=True,key=lambda x: x[1])
        return res[:5]
    except:
        return None

def print_doc(res):
    for doc in res:
    #print(doc[0])
        #id = ''
        #if doc[0] < 10:
            #id = '' + str(doc[0])
        #else:
            #id = str(doc[0])
        id=doc[0]
        id=id+1
        
        print("id document: ",id)
        id1=str(id)
        ori_doc_path = "/content/drive/MyDrive/CS521/LuatDatDai2013/Dieu"+id1+'.txt'
        #print(path)
        
        with open(ori_doc_path, 'r', encoding="utf8") as f:
            full_doc = f.read()
        print(full_doc[:200])
        print("...................................................")
        
model, idf_vector, words = load_model()
#print(words)
query = "mục 1"
print("QUERY: ", query )
query = preprocessing_query(query)
print("After processing", query)
print("------------------------------------------------\n Kết quả tìm kiếm:")
res = Cosine(model,words,idf_vector,query)
#print(res)
if res != None:
    print(res)
    print_doc(res)

QUERY:  mục 1
After processing mục 1 
------------------------------------------------
 Kết quả tìm kiếm:
[(43, 1.0), (178, 0.9999999999999999), (1, 0.3191596267486038), (11, 0.3191596267486038), (14, 0.3191596267486038)]
id document:  44
Chương 4.
QUY HOẠCH, KẾ HOẠCH SỬ DỤNG ĐẤT
Điều 44. Thẩm định quy hoạch, kế hoạch sử dụng đất
1. Thẩm quyền thành lập Hội đồng thẩm định quy hoạch, kế hoạch sử dụng đất:
a) Thủ tướng Chính phủ thành lậ
...................................................
id document:  179
Chương 11.
QUYỀN VÀ NGHĨA VỤ CỦA NGƯỜI SỬ DỤNG ĐẤT
MỤC 3. QUYỀN VÀ NGHĨA VỤ CỦA HỘ GIA ĐÌNH, CÁ NHÂN, CỘNG ĐỒNG DÂN CƯ SỬ DỤNG ĐẤT
Điều 179. Quyền và nghĩa vụ của hộ gia đình, cá nhân sử dụng đất
1. H
...................................................
id document:  2
Chương 1.
QUY ĐỊNH CHUNG
Điều 2. Đối tượng áp dụng
1. Cơ quan nhà nước thực hiện quyền hạn và trách nhiệm đại diện chủ sở hữu toàn dân về đất đai, thực hiện nhiệm vụ thống nhất quản lý nhà nước về đất
....................

In [ ]:
import pickle
from pyvi import ViTokenizer, ViPosTagger
import numpy as np

def load_model():
    file_path = ["/content/drive/MyDrive/CS521/Vector Space/model_old.pkl", "/content/drive/MyDrive/CS521/Vector Space/tfidf_old.pkl", "/content/drive/MyDrive/CS521/Vector Space/word_old.pkl"]
    ### Load Model
    with open(file_path[0], 'rb') as f:
        model = pickle.load(f)
    ### Load Idf_vector
    with open(file_path[1], 'rb') as f:
        idf_vector = pickle.load(f)
    ### Load Word
    with open(file_path[2], 'rb') as f:
        word = pickle.load(f)    
    return model, idf_vector, word

def preprocessing_query(word):
    dict_path =r'/content/drive/MyDrive/CS521/pro_dictionary.txt'
    with open(dict_path, encoding='utf8') as fobj:
	    stop_words=fobj.read()

    words = ViTokenizer.tokenize(word)
    words = words.lower()
    words = words.split()
    stop_words = stop_words.split()
    prew = ''
    for word in words:
        if word not in stop_words:
            prew = prew + word + ' '

    return prew

def Norm2(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            res.append((i,np.linalg.norm(vt-vector_query))) #
        res.sort(key=lambda x: x[1])
        return res[:5]
    except:
        return None

#Scalar
def Scalar(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            SProduct=vt@vector_query
            res.append((i,SProduct))
        res.sort(reverse=True,key=lambda x: x[1])
        return res[:5]
    except:
        return None
        
#Cosine
def Cosine(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            cos=(vt@vector_query)/(np.linalg.norm(vt)*np.linalg.norm(vector_query))
            res.append((i,cos))
        res.sort(reverse=True,key=lambda x: x[1])
        return res[:5]
    except:
        return None

def print_doc(res):
    for doc in res:
    #print(doc[0])
        #id = ''
        #if doc[0] < 10:
            #id = '' + str(doc[0])
        #else:
            #id = str(doc[0])
        id=doc[0]
        id=id+1
        
        print("id document: ",id)
        id1=str(id)
        ori_doc_path = "/content/drive/MyDrive/CS521/LuatDatDai2013/Dieu"+id1+'.txt'
        #print(path)
        
        with open(ori_doc_path, 'r', encoding="utf8") as f:
            full_doc = f.read()
        print(full_doc[:200])
        print("...................................................")
        
model, idf_vector, words = load_model()
#print(words)
query = "điều 211"
print("QUERY: ", query )
query = preprocessing_query(query)
print("After processing", query)
print("------------------------------------------------\n Kết quả tìm kiếm:")
res = Cosine(model,words,idf_vector,query)
#print(res)
if res != None:
    print(res)
    print_doc(res)

QUERY:  điều 211
After processing 211 
------------------------------------------------
 Kết quả tìm kiếm:
[(127, 1.0)]
id document:  128
Chương 10.
CHẾ ĐỘ SỬ DỤNG CÁC LOẠI ĐẤT
MỤC 1. THỜI HẠN SỬ DỤNG ĐẤT
Điều 128. Thời hạn sử dụng đất khi nhận chuyển quyền sử dụng đất
1. Thời hạn sử dụng đất khi nhận chuyển quyền sử dụng đất đối với lo
...................................................


#Test Model mới

In [ ]:
import pickle
from pyvi import ViTokenizer, ViPosTagger
import numpy as np

def load_model():
    file_path = ["/content/drive/MyDrive/CS521/Vector Space/model.pkl", "/content/drive/MyDrive/CS521/Vector Space/tfidf.pkl", "/content/drive/MyDrive/CS521/Vector Space/word.pkl"]
    ### Load Model
    with open(file_path[0], 'rb') as f:
        model = pickle.load(f)
    ### Load Idf_vector
    with open(file_path[1], 'rb') as f:
        idf_vector = pickle.load(f)
    ### Load Word
    with open(file_path[2], 'rb') as f:
        word = pickle.load(f)    
    return model, idf_vector, word

def preprocessing_query(word):
    dict_path =r'/content/drive/MyDrive/CS521/pro_dictionary (1).txt'
    with open(dict_path, encoding='utf8') as fobj:
	    stop_words=fobj.read()

    words = ViTokenizer.tokenize(word)
    words = words.lower()
    words = words.split()
    stop_words = stop_words.split()
    prew = ''
    for word in words:
        if word not in stop_words:
            prew = prew + word + ' '

    return prew

def Norm2(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            res.append((i,np.linalg.norm(vt-vector_query))) #
        res.sort(key=lambda x: x[1])
        return res
    except:
        return None

#Scalar
def Scalar(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            SProduct=vt@vector_query
            res.append((i,SProduct))
        res.sort(reverse=True,key=lambda x: x[1])
        return res
    except:
        return None
        
#Cosine
def Cosine(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            cos=(vt@vector_query)/(np.linalg.norm(vt)*np.linalg.norm(vector_query))
            res.append((i,cos))
        res.sort(reverse=True,key=lambda x: x[1])
        return res
    except:
        return None

def print_doc(res):
    for doc in res:
    #print(doc[0])
        #id = ''
        #if doc[0] < 10:
            #id = '' + str(doc[0])
        #else:
            #id = str(doc[0])
        id=doc[0]
        id=id+1
        
        print("id document: ",id)
        id1=str(id)
        ori_doc_path = "/content/drive/MyDrive/CS521/LuatDatDai2013/Dieu"+id1+'.txt'
        #print(path)
        
        with open(ori_doc_path, 'r', encoding="utf8") as f:
            full_doc = f.read()
        print(full_doc[:200])
        print("...................................................")
        
model, idf_vector, words = load_model()
#print(words)
query = "mục 1 chương 2"
print("QUERY: ", query )
query = preprocessing_query(query)
q1=query.split(" ")
query=''
for i in q1:
  if i=="mục" or i=="điều" or i=="chương":
    query=query+i+"_"
  else:
    query=query+i+" "
query=query[:-1]
print("After processing", query)
print("------------------------------------------------\n Kết quả tìm kiếm:")
res = Cosine(model,words,idf_vector,query)
#print(res)
if res != None:
    print(res)
    print_doc(res)

QUERY:  mục 1 chương 2
After processing mục_1 chương_2 
------------------------------------------------
 Kết quả tìm kiếm:
[(12, 1.0), (13, 1.0), (14, 1.0), (15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0), (22, 0.7852983555037268), (23, 0.7852983555037268), (24, 0.7852983555037268), (25, 0.7852983555037268), (26, 0.7852983555037268), (27, 0.7852983555037267), (21, 0.7852983555037266), (66, 0.6191175113362103), (94, 0.6191175113362103), (95, 0.6191175113362103), (110, 0.6191175113362103), (28, 0.6191175113362102), (29, 0.6191175113362102), (163, 0.6191175113362102), (164, 0.6191175113362102), (165, 0.6191175113362102), (166, 0.6191175113362102), (167, 0.6191175113362102), (168, 0.6191175113362102), (169, 0.6191175113362102), (64, 0.6191175113362102), (65, 0.6191175113362102), (67, 0.6191175113362102), (68, 0.6191175113362102), (69, 0.6191175113362102), (70, 0.6191175113362102), (71, 0.6191175113362102), (72, 0.6191175113362102), (73, 0.6191175113362102), (74, 0.6191175

In [ ]:
import pickle
from pyvi import ViTokenizer, ViPosTagger
import numpy as np

def load_model():
    file_path = ["/content/drive/MyDrive/CS521/Vector Space/model.pkl", "/content/drive/MyDrive/CS521/Vector Space/tfidf.pkl", "/content/drive/MyDrive/CS521/Vector Space/word.pkl"]
    ### Load Model
    with open(file_path[0], 'rb') as f:
        model = pickle.load(f)
    ### Load Idf_vector
    with open(file_path[1], 'rb') as f:
        idf_vector = pickle.load(f)
    ### Load Word
    with open(file_path[2], 'rb') as f:
        word = pickle.load(f)    
    return model, idf_vector, word

def preprocessing_query(word):
    dict_path =r'/content/drive/MyDrive/CS521/pro_dictionary (1).txt'
    with open(dict_path, encoding='utf8') as fobj:
	    stop_words=fobj.read()

    words = ViTokenizer.tokenize(word)
    words = words.lower()
    words = words.split()
    stop_words = stop_words.split()
    prew = ''
    for word in words:
        if word not in stop_words:
            prew = prew + word + ' '

    return prew

def Norm2(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            res.append((i,np.linalg.norm(vt-vector_query))) #
        res.sort(key=lambda x: x[1])
        return res
    except:
        return None

#Scalar
def Scalar(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            SProduct=vt@vector_query
            res.append((i,SProduct))
        res.sort(reverse=True,key=lambda x: x[1])
        return res
    except:
        return None
        
#Cosine
def Cosine(model,words, idf_vector,query):
    content_query=query.split()
    words_query=list(set(content_query))
    vector_query=[]
    res=[]
    try:
        for i in words_query:
            tf=content_query.count(i)/len(content_query)
            idf=idf_vector[words.index(i)]
            vector_query.append(tf*idf)
        vector_query=np.array(vector_query)

        docs_set=set()
        for i in words_query:
            docs_set.update(model[i].keys())
        docs_set=list(docs_set)
        for i in docs_set:
            vt=list()
            for j in words_query:
                vt.append(model[j].setdefault(i,0))
            vt=np.array(vt)
            cos=(vt@vector_query)/(np.linalg.norm(vt)*np.linalg.norm(vector_query))
            res.append((i,cos))
        res.sort(reverse=True,key=lambda x: x[1])
        return res
    except:
        return None

def print_doc(res):
    for doc in res:
    #print(doc[0])
        #id = ''
        #if doc[0] < 10:
            #id = '' + str(doc[0])
        #else:
            #id = str(doc[0])
        id=doc[0]
        
        id=id+1
        
        print("id document: ",id)
        id1=str(id)
        ori_doc_path = "/content/drive/MyDrive/CS521/LuatDatDai2013/Dieu"+id1+'.txt'
        #print(path)
        
        with open(ori_doc_path, 'r', encoding="utf8") as f:
            full_doc = f.read()
        print(full_doc[:200])
        print("...................................................")
        
model, idf_vector, words = load_model()
#print(words)
query = "điều 125"
print("QUERY: ", query )
query = preprocessing_query(query)
q1=query.split(" ")
query=''
for i in q1:
  if i=="mục" or i=="điều" or i=="chương":
    query=query+i+"_"
  else:
    query=query+i+" "
query=query[:-1]
print("After processing", query)
print("------------------------------------------------\n Kết quả tìm kiếm:")
res = Cosine(model,words,idf_vector,query)
#print(res)
if res != None:
    print(res)
    print_doc(res)

QUERY:  điều 125
After processing điều_125 
------------------------------------------------
 Kết quả tìm kiếm:
[(127, 1.0)]
id document:  128
Chương 10.
CHẾ ĐỘ SỬ DỤNG CÁC LOẠI ĐẤT
MỤC 1. THỜI HẠN SỬ DỤNG ĐẤT
Điều 128. Thời hạn sử dụng đất khi nhận chuyển quyền sử dụng đất
1. Thời hạn sử dụng đất khi nhận chuyển quyền sử dụng đất đối với lo
...................................................
